In [38]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [39]:
df = pd.read_excel('Faktoren Wartezeit.xlsx')

In [40]:
# trim all columns names
df.columns = df.columns.str.strip()
# strip all values
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

In [41]:
# set index to Identifationsnummer
df.set_index('Identifikationsnummer', inplace=True)

In [42]:
# set Geburtsdatum as datetime format and calculate age in years
df['Geburtsdatum'] = pd.to_datetime(df['Geburtsdatum'])
df['Alter'] = (pd.to_datetime('today') - df['Geburtsdatum']).dt.days // 365

In [43]:
# calculate Wartezeit in days
df['Wartezeit in Tagen'] = (df['Diagnostiktermin'] - df['Anmeldedatum']).dt.days

In [44]:
df

,Geburtsdatum,Alter,Geschlecht,Störungsgebiet,Anmeldedatum,Diagnostiktermin,Wartezeit in Tagen,Montag,Dienstag,Mittwoch,Donnerstag,Freitag,Versichert,Verordnung,Unnamed: 15
Identifikationsnummer,,,,,,,,,,,,,,,
66,2018-05-10,6,weiblich,SP1,2024-02-12,2024-04-09,57,"nachmittags, abends","nachmittags, abends","nachmittags, abends",unverfügbar,"nachmittags, abends",gesetzlich,NaN,NaN
90,2017-12-19,6,männlich,RE1,2023-02-12,2023-08-17,186,"nachmittags, abends","nachmittags, abends","nachmittags, abends","nachmittags, abends","nachmittags, abends",gesetzlich,NaN,NaN
137,2015-09-22,8,männlich,SP1,2022-11-24,2023-02-10,78,unverfügbar,unverfügbar,unverfügbar,"nachmittags, abends","nachmittags, abends",gesetzlich,Vda 22.11.2024,NaN
152,2018-11-07,5,männlich,SP3,2024-03-11,2024-04-15,35,"nachmittags, abends",unverfügbar,unverfügbar,"nachmittags, abends","nachmittags, abends",gesetzlich,NaN,NaN
193,2017-08-05,6,weiblich,SP1,2023-05-04,2023-09-15,134,"morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,2019-09-22,4,männlich,SP3,2024-03-19,2024-04-30,42,"morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags, mittags, nachmittags, abends",gesetzlich,NaN,NaN
390,2019-02-05,5,weiblich,SP3,2024-04-09,2023-04-25,-350,"morgens, vormittags","morgens, vormittags, mittags, nachmittags, abends","morgens, vormittags","morgens, vormittags","nachmittags, abends",gesetzlich,Vda wird,NaN
391,2016-03-10,8,männlich,SP1,2024-03-11,2024-04-22,42,"nachmittags, abends",unverfügbar,"nachmittags, abends","nachmittags, abends","nachmittags, abends",gesetzlich,Vda 26.02.2024,NaN


In [45]:
# check column Montag and create inner list with values of row
df["Montag"] = df["Montag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Dienstag"] = df["Dienstag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Mittwoch"] = df["Mittwoch"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Donnerstag"] = df["Donnerstag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])
df["Freitag"] = df["Freitag"].apply(lambda x: x.split(', ') if pd.notna(x) else ["unverfügbar"])

In [46]:
df

,Geburtsdatum,Alter,Geschlecht,Störungsgebiet,Anmeldedatum,Diagnostiktermin,Wartezeit in Tagen,Montag,Dienstag,Mittwoch,Donnerstag,Freitag,Versichert,Verordnung,Unnamed: 15
Identifikationsnummer,,,,,,,,,,,,,,,
66,2018-05-10,6,weiblich,SP1,2024-02-12,2024-04-09,57,"[nachmittags, abends]","[nachmittags, abends]","[nachmittags, abends]",[unverfügbar],"[nachmittags, abends]",gesetzlich,NaN,NaN
90,2017-12-19,6,männlich,RE1,2023-02-12,2023-08-17,186,"[nachmittags, abends]","[nachmittags, abends]","[nachmittags, abends]","[nachmittags, abends]","[nachmittags, abends]",gesetzlich,NaN,NaN
137,2015-09-22,8,männlich,SP1,2022-11-24,2023-02-10,78,[unverfügbar],[unverfügbar],[unverfügbar],"[nachmittags, abends]","[nachmittags, abends]",gesetzlich,Vda 22.11.2024,NaN
152,2018-11-07,5,männlich,SP3,2024-03-11,2024-04-15,35,"[nachmittags, abends]",[unverfügbar],[unverfügbar],"[nachmittags, abends]","[nachmittags, abends]",gesetzlich,NaN,NaN
193,2017-08-05,6,weiblich,SP1,2023-05-04,2023-09-15,134,"[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,2019-09-22,4,männlich,SP3,2024-03-19,2024-04-30,42,"[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags, mittags, nachmittags, ab...",gesetzlich,NaN,NaN
390,2019-02-05,5,weiblich,SP3,2024-04-09,2023-04-25,-350,"[morgens, vormittags]","[morgens, vormittags, mittags, nachmittags, ab...","[morgens, vormittags]","[morgens, vormittags]","[nachmittags, abends]",gesetzlich,Vda wird,NaN
391,2016-03-10,8,männlich,SP1,2024-03-11,2024-04-22,42,"[nachmittags, abends]",[unverfügbar],"[nachmittags, abends]","[nachmittags, abends]","[nachmittags, abends]",gesetzlich,Vda 26.02.2024,NaN


In [47]:

df_dummies_montag = pd.get_dummies(df['Montag'].apply(pd.Series).stack(), prefix='Montag', dummy_na=False)
df_dummies_dienstag = pd.get_dummies(df['Dienstag'].apply(pd.Series).stack(), prefix='Dienstag', dummy_na=False)
df_dummies_mittwoch = pd.get_dummies(df['Mittwoch'].apply(pd.Series).stack(), prefix='Mittwoch', dummy_na=False)
df_dummies_donnerstag = pd.get_dummies(df['Donnerstag'].apply(pd.Series).stack(), prefix='Donnerstag', dummy_na=False)
df_dummies_freitag = pd.get_dummies(df['Freitag'].apply(pd.Series).stack(), prefix='Freitag', dummy_na=False)


In [48]:
df_dummies_montag_grouped = df_dummies_montag.groupby(level=0).sum()
df_dummies_dienstag_grouped = df_dummies_dienstag.groupby(level=0).sum()
df_dummies_mittwoch_grouped = df_dummies_mittwoch.groupby(level=0).sum()
df_dummies_donnerstag_grouped = df_dummies_donnerstag.groupby(level=0).sum()
df_dummies_freitag_grouped = df_dummies_freitag.groupby(level=0).sum()

In [49]:
df_dummies_versichert = pd.get_dummies(df['Versichert'], prefix='Versichert')
df_dummies_stoerungsgebiet = pd.get_dummies(df['Störungsgebiet'], prefix='Störungsgebiet')
df_dummies_geschlecht = pd.get_dummies(df['Geschlecht'], prefix='Geschlecht')
df["Alter"] = pd.cut(df["Alter"], bins=[0, 4, 12, 18, 67], labels=["0-4", "4-12", "12-18", "18-67"])
df_dummies_alter = pd.get_dummies(df['Alter'], prefix='Alter')

In [50]:
# drop all columns that are used for dummies
df.drop(['Alter', 'Geburtsdatum', 'Geschlecht', 'Anmeldedatum', 'Diagnostiktermin', 'Störungsgebiet', 'Montag', 'Dienstag', 'Mittwoch', 'Donnerstag', 'Freitag', 'Versichert', 'Verordnung', 'Unnamed: 15'], axis=1, inplace=True)

In [51]:
df.sort_index(inplace=True)

In [52]:
# merge all dummies
df = pd.concat([df, df_dummies_alter, df_dummies_geschlecht, df_dummies_montag_grouped, df_dummies_dienstag_grouped, df_dummies_mittwoch_grouped, df_dummies_donnerstag_grouped, df_dummies_freitag_grouped, df_dummies_versichert, df_dummies_stoerungsgebiet], axis=1)

In [59]:
df = df.astype(int)

In [60]:
df

,Wartezeit in Tagen,Alter_0-4,Alter_4-12,Alter_12-18,Alter_18-67,Geschlecht_männlich,Geschlecht_weiblich,Montag_abends,Montag_mittags,Montag_morgens,...,Versichert_gestzlich,Versichert_privat,Störungsgebiet_OFD,Störungsgebiet_OFZ,Störungsgebiet_RE1,Störungsgebiet_SCZ,Störungsgebiet_SP1,Störungsgebiet_SP3,Störungsgebiet_SP5,Störungsgebiet_SPZ
Identifikationsnummer,,,,,,,,,,,,,,,,,,,,,
66,57,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
90,186,0,1,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
137,78,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
152,35,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
193,134,0,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,42,1,0,0,0,1,0,1,1,1,...,0,0,0,0,0,0,0,1,0,0
390,-350,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
391,42,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [61]:
df.to_excel('df_final.xlsx')

In [55]:
df.head()

,Wartezeit in Tagen,Alter_0-4,Alter_4-12,Alter_12-18,Alter_18-67,Geschlecht_männlich,Geschlecht_weiblich,Montag_abends,Montag_mittags,Montag_morgens,...,Versichert_gestzlich,Versichert_privat,Störungsgebiet_OFD,Störungsgebiet_OFZ,Störungsgebiet_RE1,Störungsgebiet_SCZ,Störungsgebiet_SP1,Störungsgebiet_SP3,Störungsgebiet_SP5,Störungsgebiet_SPZ
Identifikationsnummer,,,,,,,,,,,,,,,,,,,,,
66,57,0,1,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
90,186,0,1,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
137,78,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
152,35,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
193,134,0,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0


In [56]:
df.columns.tolist()

['Wartezeit in Tagen',
 'Alter_0-4',
 'Alter_4-12',
 'Alter_12-18',
 'Alter_18-67',
 'Geschlecht_männlich',
 'Geschlecht_weiblich',
 'Montag_abends',
 'Montag_mittags',
 'Montag_morgens',
 'Montag_nachmittags',
 'Montag_unverfügbar',
 'Montag_vormittags',
 'Dienstag_abends',
 'Dienstag_mittags',
 'Dienstag_morgens',
 'Dienstag_nachmittags',
 'Dienstag_unverfügbar',
 'Dienstag_vormittags',
 'Mittwoch_abends',
 'Mittwoch_mittags',
 'Mittwoch_morgens',
 'Mittwoch_nachmittags',
 'Mittwoch_unverfügbar',
 'Mittwoch_vormittags',
 'Donnerstag_abends',
 'Donnerstag_mittags',
 'Donnerstag_morgens',
 'Donnerstag_nachmittags',
 'Donnerstag_unverfügbar',
 'Donnerstag_vormittags',
 'Freitag_abends',
 'Freitag_mittags',
 'Freitag_morgens',
 'Freitag_nachmittags',
 'Freitag_unverfügbar',
 'Freitag_vormittags',
 'Versichert_gesetzlich',
 'Versichert_gestzlich',
 'Versichert_privat',
 'Störungsgebiet_OFD',
 'Störungsgebiet_OFZ',
 'Störungsgebiet_RE1',
 'Störungsgebiet_SCZ',
 'Störungsgebiet_SP1',
 'Stör

In [57]:
df.to_excel('df_final.xlsx')

In [62]:
df.drop(columns=['Alter_12-18', 'Alter_18-67'], inplace=True)

In [72]:
corr = df.corr().round(3)
corr = corr.style.background_gradient(cmap='RdYlGn')

In [73]:
corr

,Wartezeit in Tagen,Alter_0-4,Alter_4-12,Geschlecht_männlich,Geschlecht_weiblich,Montag_abends,Montag_mittags,Montag_morgens,Montag_nachmittags,Montag_unverfügbar,Montag_vormittags,Dienstag_abends,Dienstag_mittags,Dienstag_morgens,Dienstag_nachmittags,Dienstag_unverfügbar,Dienstag_vormittags,Mittwoch_abends,Mittwoch_mittags,Mittwoch_morgens,Mittwoch_nachmittags,Mittwoch_unverfügbar,Mittwoch_vormittags,Donnerstag_abends,Donnerstag_mittags,Donnerstag_morgens,Donnerstag_nachmittags,Donnerstag_unverfügbar,Donnerstag_vormittags,Freitag_abends,Freitag_mittags,Freitag_morgens,Freitag_nachmittags,Freitag_unverfügbar,Freitag_vormittags,Versichert_gesetzlich,Versichert_gestzlich,Versichert_privat,Störungsgebiet_OFD,Störungsgebiet_OFZ,Störungsgebiet_RE1,Störungsgebiet_SCZ,Störungsgebiet_SP1,Störungsgebiet_SP3,Störungsgebiet_SP5,Störungsgebiet_SPZ
Wartezeit in Tagen,1.000000,-0.034000,0.049000,0.064000,-0.064000,0.102000,-0.038000,-0.133000,0.048000,0.030000,-0.109000,0.156000,-0.142000,-0.118000,0.076000,-0.150000,-0.119000,0.058000,-0.019000,-0.116000,0.123000,0.086000,-0.091000,0.212000,-0.026000,-0.129000,0.226000,-0.065000,-0.118000,-0.065000,-0.179000,-0.180000,-0.072000,0.074000,-0.051000,0.041000,-0.021000,-0.055000,0.009000,-0.017000,0.049000,0.069000,0.133000,-0.174000,-0.002000,-0.042000
Alter_0-4,-0.034000,1.000000,-0.929000,0.133000,-0.133000,0.053000,-0.034000,0.087000,0.134000,-0.035000,0.016000,0.136000,0.100000,0.144000,0.197000,-0.130000,0.087000,0.161000,0.043000,0.153000,0.203000,-0.130000,0.087000,0.016000,-0.049000,0.043000,0.069000,0.030000,-0.018000,0.053000,0.010000,0.137000,0.092000,-0.016000,0.060000,0.044000,0.363000,-0.130000,-0.035000,-0.035000,-0.071000,-0.035000,-0.034000,0.128000,-0.035000,-0.050000
Alter_4-12,0.049000,-0.929000,1.000000,-0.167000,0.167000,0.031000,0.062000,-0.055000,-0.070000,-0.053000,0.009000,-0.147000,-0.059000,-0.105000,-0.120000,0.140000,-0.055000,-0.173000,-0.009000,-0.115000,-0.218000,0.140000,-0.055000,0.079000,0.078000,-0.009000,0.010000,-0.145000,0.045000,0.031000,0.025000,-0.095000,-0.019000,-0.079000,-0.026000,-0.063000,-0.338000,0.140000,0.038000,0.038000,0.076000,0.038000,-0.000000,-0.096000,0.038000,0.053000
Geschlecht_männlich,0.064000,0.133000,-0.167000,1.000000,-1.000000,-0.031000,-0.026000,0.055000,0.052000,0.032000,0.028000,-0.102000,0.076000,-0.000000,-0.028000,0.140000,0.000000,0.153000,0.026000,-0.026000,0.055000,0.000000,-0.055000,-0.034000,0.026000,0.026000,-0.029000,0.040000,0.027000,0.031000,-0.025000,-0.110000,0.056000,-0.034000,-0.079000,0.187000,-0.113000,-0.140000,-0.113000,0.113000,0.229000,-0.113000,0.155000,-0.231000,-0.113000,0.000000
Geschlecht_weiblich,-0.064000,-0.133000,0.167000,-1.000000,1.000000,0.031000,0.026000,-0.055000,-0.052000,-0.032000,-0.028000,0.102000,-0.076000,-0.000000,0.028000,-0.140000,0.000000,-0.153000,-0.026000,0.026000,-0.055000,0.000000,0.055000,0.034000,-0.026000,-0.026000,0.029000,-0.040000,-0.027000,-0.031000,0.025000,0.110000,-0.056000,0.034000,0.079000,-0.187000,0.113000,0.140000,0.113000,-0.113000,-0.229000,0.113000,-0.155000,0.231000,0.113000,0.000000
Montag_abends,0.102000,0.053000,0.031000,-0.031000,0.031000,1.000000,0.306000,0.207000,0.708000,-0.925000,0.171000,0.103000,0.187000,0.189000,0.210000,-0.124000,0.207000,0.328000,0.201000,0.242000,0.260000,-0.381000,0.207000,0.185000,0.201000,0.201000,0.213000,-0.105000,0.218000,0.477000,0.249000,0.207000,0.364000,-0.351000,0.242000,0.046000,-0.217000,-0.039000,0.058000,0.058000,-0.161000,-0.217000,0.039000,0.018000,0.058000,0.083000
Montag_mittags,-0.038000,-0.034000,0.062000,-0.026000,0.026000,0.306000,1.000000,0.686000,0.413000,-0.343000,0.716000,0.171000,0.583000,0.529000,0.278000,-0.152000,0.571000,0.177000,0.782000,0.553000,0.237000,-0.152000,0.571000,0.171000,0.782000,0.562000,0.212000,-0.170000,0.602000,0.112000,0.691000,0.489000,0.084000,-0.099000,0.609000,-0.040000,-0.080000,0.070000,-0.080000,-0.080000,-0.042000,-0.080000,-0.048000,0.076000,-0.080000,0.224000
Monta

In [79]:
import dataframe_image as dfi
dfi.export(corr, 'corr.png', max_cols=-1)